In [ ]:
import os
from config_file import config
import sys
#need to be change in function of your own sys path
sys.path.append("/export/home/nidebroux/sct_custom")
sys.path.append("/export/home/nidebroux/modules")

import shutil
import numpy as np
import pandas as pd

import spinalcordtoolbox.utils as sct
from spinalcordtoolbox.image import Image
from spinalcordtoolbox.deepseg_sc.core import crop_image_around_centerline 
from spinalcordtoolbox.deepseg_sc.core import apply_intensity_normalization
from spinalcordtoolbox.deepseg_sc.core import find_centerline

main_dir = config['main_dir']

In [ ]:
# Specifying input data
PATH_nii= main_dir + 'data_mask/nii/'
PATH_mask =  main_dir + 'data_mask/mask/'
subject_masks=os.listdir(PATH_mask)
Mask_list=[x.split('.')[0] for x in subject_masks]
Mask_list.remove('unused')
Mask_list.remove('')
print(Mask_list)

for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]

In [ ]:
# Reorientation of original image and ROI

for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    contrast = Type[1]
    img_path = PATH_nii + subject + '_'+ contrast + '.nii'    
    print(img_path)   
    sct.run_proc(['sct_image', '-i', img_path, '-setorient', 'RPI'], verbose=0)




# Here specify LESION MASK DATA
for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    contrast = Type[1]
    img_path = PATH_mask + subject + '_'+ contrast + '_seg.nii'
    print(img_path)   
    sct.run_proc(['sct_image', '-i', img_path,  '-setorient','RPI'], verbose=0)



In [ ]:
# Resampling of image

# Resample Input image
out_path =  main_dir + 'data_mask/preprocess/Resample/'
 
for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    contrast = Type[1]
    img_path = PATH_nii + subject + '_' +contrast + '.nii'
    img_out_path = out_path + subject + '_'+contrast +'_resample_INPUT.nii.gz'   
    print(img_out_path)
    im = Image(img_path)
    mm3 = str(im.dim[6])
    sct.run_proc(['sct_resample', '-i', img_path, '-x', 'nn', '-mm', '0.5x0.5x'+mm3, '-o', img_out_path], verbose=0)
      


        
# Resample mask

for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    contrast = Type[1]
    img_path = PATH_mask + subject +'_' + contrast +'_seg.nii'
    img_out_path = out_path + subject + '_' +contrast + '_resample_seg.nii.gz'   
    print(img_out_path)
    im = Image(img_path)
    mm3 = str(im.dim[6])
    sct.run_proc(['sct_resample', '-i', img_path, '-x', 'nn', '-mm', '0.5x0.5x'+mm3, '-o', img_out_path], verbose=0)


In [ ]:
#centerline detection
PATH_resample =  main_dir + 'data_mask/preprocess/Resample/' 


#exact centerline

for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    contrast = Type[1]
    img_path = PATH_resample + subject + '_'+ contrast + '_resample_seg.nii.gz'
    target_path = PATH_resample + subject +  '_'+ contrast+ '_resample_centerline'
    print(img_path) 
    sct.run_proc(['sct_get_centerline', '-i', img_path,'-c',contrast ,'-o',target_path,'-method','fitseg','-igt',img_path], verbose=0)
   



In [ ]:
# 5.0 Cropping image

out_path =  main_dir + 'data_mask/preprocess/crop/'
PATH_resample =  main_dir + 'data_mask/preprocess/Resample/'


for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    contrast = Type[1]
    
    NII_image = Image(PATH_resample + subject +  '_'+ contrast + '_resample_INPUT.nii.gz')
    MASK_image = Image(PATH_resample + subject +  '_'+ contrast + '_resample_seg.nii.gz')
    ctr_image = Image(PATH_resample + subject +  '_'+ contrast + '_resample_centerline.nii.gz')
    NII_image.loadFromPath(path = PATH_resample + subject +  '_'+ contrast + '_resample_INPUT.nii.gz',verbose=1)
    MASK_image.loadFromPath(path = PATH_resample + subject +  '_'+ contrast + '_resample_seg.nii.gz',verbose=1)
    ctr_image.loadFromPath(path = PATH_resample + subject +  '_'+ contrast + '_resample_centerline.nii.gz',verbose=1)  
    cnn_ctr_image.loadFromPath(path = PATH_resample + subject +  '_'+ contrast + '_resample_cnn_centerline.nii.gz',verbose=1) 
    
    img_NII_path = out_path + subject +  '_'+ contrast + '_crop_INPUT.nii.gz'
    img_MASK_path = out_path + subject +  '_'+ contrast + '_crop_SEG.nii.gz'     
    nii_crop = crop_image_around_centerline(NII_image,ctr_image, 64)
    seg_crop = crop_image_around_centerline(MASK_image, ctr_image, 64)
    nii_crop[3].save(img_NII_path)
    seg_crop[3].save(img_MASK_path)
  

    

In [ ]:
# Standardize image intensities
ofolder_path = main_dir + 'data_mask/preprocess/Norm_int/'
path_crop = main_dir + 'data_mask/preprocess/crop/'
for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    contrast =   Type[1]
    img_in_path = Image(path_crop + subject +   '_'+ contrast +'_crop_INPUT.nii.gz')
    img_in_path.loadFromPath(path =path_crop + subject +   '_'+ contrast +'_crop_INPUT.nii.gz',verbose=1 )
    img_out_path = ofolder_path + subject +  '_'+ contrast + '_norm.nii.gz'
    img = apply_intensity_normalization(img_in_path)  
    img.save(img_out_path)
    


In [ ]:
# Save as Pickle file
DATA_PATH =  main_dir + 'data_mask/preprocess/Data_organized/' #% where the data is organized
path_crop =  main_dir + 'data_mask/preprocess/crop/'
path_norm =  main_dir + 'data_mask/preprocess/Norm_int/'

subjectlist = []
contrastlist = []
for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    print(subject)
    subjectlist.append(subject)
    contrast = Type[1]
    contrastlist.append(contrast)
    new_dir =DATA_PATH + subject + '/' + contrast + '/' 
    #comment line below if drection already exists
    #os.makedirs(new_dir)
    shutil.copy(path_crop + subject +  '_'+ contrast + '_crop_cnn_SEG.nii.gz', new_dir)
    shutil.copy(path_norm + subject +  '_'+ contrast + '_norm.nii.gz', new_dir)

    
    



original_df = pd.DataFrame({"subject": subjectlist, "contrast_foldname": contrastlist})
original_df.to_pickle(main_dir + "data_mask/subject_dict.pkl")